In this notebook, I will transform the datasets I will use for this project by eliminating unuseful columns and rows, correcting certain post classification, changing column and value names, etc.

In [1]:
import pandas as pd
!pip install emoji
!pip install tweepy
import tweepy
import emoji
import re
import numpy
import warnings
#In order to ignore warnings when overwriting row values in panda dataframes
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 8.9 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=b609e6abe1bffbcb1c1f63851554b84859606e9b47745f07196718643c0fd6cc
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


####Tweepy credentials

In [47]:
CONSUMER_KEY = #
CONSUMER_SECRET = #
OAUTH_TOKEN = #
OAUTH_TOKEN_SECRET = #

In [48]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

####Imports & functions

In [3]:
#AMI dataset's emojis don't show as emojis, we need to replace the emoji code with the emoji in the df
df_emoji_code_to_emoji = pd.read_csv('/content/emoji_codes_csv.csv', encoding='utf8', engine='python')
df_emoji_code_to_emoji = df_emoji_code_to_emoji[["hex code", "emoji_code", "emoji"]]

In [34]:
#defining functions
def unique(list1):
    x = numpy.array(list1)
    return numpy.unique(x)

#If a hashtag is followed by another hashtag or a special character (?!), this divides it
def divideHashtagtext(txt):
  specialChars = "#”.,@?¿¡!|:" 
  for specialChar in specialChars:
    txt = txt.replace(specialChar, "$$"+specialChar)
  txt = txt.replace(',', ' ')
  return txt.split("$$")

#Deletes spanish accents in a string
def deleteAccents(txt):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        txt = txt.replace(a, b).replace(a.upper(), b.upper())
    return txt

#Returns all emojis in a string
def check_emojis(tweet : str):
  words = list(tweet)
  listEmojis = str()
  for i in words:
    if emoji.is_emoji(i):
      listEmojis = listEmojis + " " + i
  return listEmojis

#Returns all codes with the structure &#____; (indicating emojis/special symbols)
def getAllEmojiHexCode(df):
  emoji_codes = []
  pattern = "&#(.*?)\;"
  for idx, row in df.iterrows():
    match = re.findall(pattern, row.text)
    for m in match:
      emoji_codes.append( "&#" + m + ";")

  return unique(emoji_codes)

#Replaces hex codes with the corresponding emoji/symbol in a dataframe
def replaceEmojiCodes(df):
  for idx, row in df.iterrows():
    for eidx, erow in df_emoji_code_to_emoji.iterrows():
      row.text = row.text.replace(df_emoji_code_to_emoji['hex code'][eidx], str(df_emoji_code_to_emoji['emoji'][eidx])) 
      df['text'][idx] = row.text

  return df

#Returns exception message (for Tweepy tweet extraction)
def getExceptionMessage(msg):
    words = msg.split(' ')
    errorMsg = ""
    for index, word in enumerate(words):
        if index not in [0,1,2]:
            errorMsg = errorMsg + ' ' + word
    errorMsg = errorMsg.rstrip("\'}]")
    errorMsg = errorMsg.lstrip(" \'")
    return errorMsg

#Returns keys of a dictionary whose value is superior to "num"
def important_values(dict_, num):
  dict_important = {}
  for (key, value) in dict_.items():
    if value > num:
        dict_important[key] = value
  return dict_important

#Use recursion to return all parent keys of indicated key to search in nested dictionary
def find_parent_classes(d, value):
  all_classes = []
  for k,v in d.items():
    if k == value:
          all_classes = all_classes + [k]
    elif isinstance(v, dict):
          p = find_parent_classes(v, value)
          if p:
              all_classes = all_classes + [k] + p
  return all_classes 

#Replace all emojis in keys_rep in dataframe for text in between colons, and delete al emojis in keys_del in same dataframe
def replace_emojis(keys_rep, keys_del, df):
  for idx, row in df.iterrows():
    for k in keys_del:
      row.text = row.text.replace(k, "")
      df['text'][idx] = row.text
    for k in keys_rep:
      row.text = row.text.replace(k, " " + emoji.demojize(k) + " ")
      df['text'][idx] = row.text

  return df

In [5]:
#This function returns a cleaned dataframe, plus dictionaries with all the hashtags and emojis in them
def cleanTweets_getDicts(df):
  #Just choose spanish tweets
  hashtag_dict = {}
  emoji_dict = {}
  #Change urls and mentions for a single word
  for idx, row in df.iterrows():
        #Remove ""
        row.text = row.text.strip('"')
        df['text'][idx] = row.text
        #
        all_emojis = check_emojis(row.text)
        all_ats = list(filter(lambda word: word[0]=='@', row.text.split())) 
        all_url = list(filter(lambda word: word[:8]=='https://', row.text.split())) + list(filter(lambda word: word[:7]=='http://', row.text.split()))
        all_hashtags = list(filter(lambda word: word[0]=='#', row.text.split()))   
        ##Deletes indication that the tweet is a Retweet
        if row.text.startswith('RT '):
          row.text = row.text[3:];
          df['text'][idx] = row.text
        ##
        for i in range(len(all_hashtags)):
            all_hashtags[i] = all_hashtags[i].lower() 
            if len(divideHashtagtext(all_hashtags[i])) <= 2:
              if deleteAccents(all_hashtags[i]) in hashtag_dict:
                hashtag_dict[deleteAccents(all_hashtags[i])] = hashtag_dict[deleteAccents(all_hashtags[i])] + 1
              else:
                hashtag_dict[deleteAccents(all_hashtags[i])] = 1
            else:
              hashtag_aux_list = divideHashtagtext(all_hashtags[i])
              hashtag_aux_list = hashtag_aux_list.pop()
              for string in hashtag_aux_list:
                if string[0] == '#':
                  if deleteAccents(string) in hashtag_dict:
                    hashtag_dict[deleteAccents(string)] = hashtag_dict[deleteAccents(string)] + 1
                  else:
                    hashtag_dict[deleteAccents(string)] = 1
                elif string[0] == '@':
                  row.text = row.text.replace(string, "") #MENTION
                  df['text'][idx] = row.text
        for mention in all_ats:
          row.text = row.text.replace(mention, "") #MENTION
          df['text'][idx] = row.text
        for url in all_url:
          row.text = row.text.replace(url, "") #URL
          df['text'][idx] = row.text
        for e in check_emojis(row.text):
            if e in emoji_dict:
              emoji_dict[e] = emoji_dict[e] + 1
            else:
              emoji_dict[e] = 1
              
  return df, hashtag_dict, emoji_dict

#SPANISH DATA UNIFICATION

Import all csv files that will be processed to obtain final corpus

In [7]:
#hateval
df_hateval_train = pd.read_csv('/content/hateval2019_es_train.csv', encoding='utf8')
df_hateval_test = pd.read_csv('/content/hateval2019_es_test.csv', encoding='utf8')
df_hateval_val = pd.read_csv('/content/hateval2019_es_dev.csv', encoding='utf8')

#EXIST
df_exist_train = pd.read_csv('/content/EXIST2021_training.csv', encoding='utf8', engine='python')
df_exist_test = pd.read_csv('/content/EXIST2021_test_labeled.csv', encoding='utf8', engine='python')


First of all, we'll delete and change columns in order to only have the two key aspects of these tweets: text and sexism value (0,1)

###[HATEVAL](https://aclanthology.org/S19-2007.pdf)

In [8]:
hateval_data = "HATEVAL DATASET:\n------------------------------------------\n"

In [9]:
hateval_data = hateval_data + "hatEval train set:\n"
#We won't use the columns that determined aggresiveness or target (group or individual) -> case is the same in all 3 subsets
df_hateval_train = df_hateval_train.drop(['TR', 'AG'], axis=1)
hateval_data = hateval_data + "Number of Spanish tweets: " + str(df_hateval_train.shape[0]) + "\n\n"
df_hateval_train = df_hateval_train[['text','HS']]
df_hateval_train.rename(columns = {'HS':'hate speech'}, inplace = True)

In [10]:
hateval_data = hateval_data + "hatEval test set:\n"
hateval_data = hateval_data + "Number of Spanish tweets: " + str(df_hateval_test.shape[0]) + "\n\n"
df_hateval_test = df_hateval_test[['text','HS']]
df_hateval_test.rename(columns = {'HS':'hate speech'}, inplace = True)

In [11]:
hateval_data = hateval_data + "hatEval validation set:\n"
df_hateval_val = df_hateval_val.drop(['TR', 'AG'], axis=1)
hateval_data = hateval_data + "Number of Spanish tweets: " + str(df_hateval_val.shape[0]) + "\n\n"
df_hateval_val = df_hateval_val[['text','HS']]
df_hateval_val.rename(columns = {'HS':'hate speech'}, inplace = True)

In [12]:
df_hateval = pd.concat([df_hateval_train, df_hateval_val, df_hateval_test])

In [13]:
df_hateval

,text,hate speech
0,Easyjet quiere duplicar el número de mujeres p...,1
1,El gobierno debe crear un control estricto de ...,1
2,Yo veo a mujeres destruidas por acoso laboral ...,0
3,"— Yo soy respetuoso con los demás, sólamente l...",0
4,Antonio Caballero y como ser de mal gusto e ig...,0
...,...,...
1595,@kaitedo2523 Hasta que los pongan en sus puest...,1
1596,Y una valla de seis metros con concertinas en ...,1
1597,@ArgAcuerdo Tenes razon. Pero muy distintos er...,1
1598,@07c05c81f52d432 @eeu_11 Aquí todos los delinc...,1


In [14]:
hateval_data = hateval_data + "hatEval all posts:\n"
hateval_data = hateval_data + "Number of Spanish tweets: " + str(df_hateval.shape[0]) + "\n"
hateval_data = hateval_data + "Percentage of hate speech: " + str((len(df_hateval.loc[df_hateval['hate speech'] == 1].index)*100)/df_hateval.shape[0]) + "%\n"
print(hateval_data)

HATEVAL DATASET:
------------------------------------------
hatEval train set:
Number of Spanish tweets: 4500

hatEval test set:
Number of Spanish tweets: 1600

hatEval validation set:
Number of Spanish tweets: 500

hatEval all posts:
Number of Spanish tweets: 6600
Percentage of hate speech: 41.5%



###[EXIST](http://journal.sepln.org/sepln/ojs/ojs/index.php/pln/article/viewFile/6389/3809)

In [15]:
exist_data = "EXIST DATASET:\n------------------------------------------\n"

In [16]:
exist_data = exist_data + "EXIST train set:\n"
exist_data = exist_data + "Number of tweets: " + str(df_exist_train.shape[0]) + "\n"
#Choose corresponding language
df_exist_train = df_exist_train[df_exist_train['language']=="es"]
exist_data = exist_data + "Number of Spanish tweets: " + str(df_exist_train.shape[0]) + "\n\n"
#We just use the first task (binary value: sexist or non-sexist)
df_exist_train['task1'] = df_exist_train['task1'].replace(['non-sexist'],0)
df_exist_train['task1'] = df_exist_train['task1'].replace(['sexist'],1)
df_exist_train = df_exist_train[['text','task1']]
df_exist_train.rename(columns = {'task1':'hate speech'}, inplace = True)

In [17]:
exist_data = exist_data + "EXIST test set:\n"
exist_data = exist_data + "Number of tweets: " + str(df_exist_test.shape[0]) + "\n"
#Choose corresponding language
df_exist_test = df_exist_test[df_exist_test['language']=="es"]
exist_data = exist_data + "Number of Spanish tweets: " + str(df_exist_test.shape[0]) + "\n\n"
#We just use the first task (binary value: sexist or non-sexist)
df_exist_test['task1'] = df_exist_test['task1'].replace(['non-sexist'],0)
df_exist_test['task1'] = df_exist_test['task1'].replace(['sexist'],1)
df_exist_test = df_exist_test[['text','task1']]
df_exist_test.rename(columns = {'task1':'hate speech'}, inplace = True)

In [18]:
df_exist = pd.concat([df_exist_train, df_exist_test])

In [19]:
df_exist

,text,hate speech
3436,Nadie te va a tratar tan bien como un hombre q...,1
3437,"@lindagisela74 Que rica putita obediente, afor...",1
3438,@BicireporteraDF Yo lo hice a los 18 años por ...,0
3439,las cosas q sueño son indicios de que yo enrea...,0
3440,"Pero a la niña le gustó desde que lo vió, así ...",0
...,...,...
4363,"@IreneMontero Se llama nota de corte, y es lo ...",0
4364,@freckles887 Osea todo atack of titan parte de...,0
4365,@alabadomango Cuéntame más!!Es por androcentri...,1
4366,Que duro es ser tan atractiva como Jaba de Hutt.,1


In [20]:
exist_data = exist_data + "EXIST all posts:\n"
exist_data = exist_data + "Number of Spanish tweets: " + str(df_exist.shape[0]) + "\n"
exist_data = exist_data + "Percentage of hate speech: " + str((len(df_exist.loc[df_exist['hate speech'] == 1].index)*100)/df_exist.shape[0]) + "%\n"
print(exist_data)

EXIST DATASET:
------------------------------------------
EXIST train set:
Number of tweets: 6977
Number of Spanish tweets: 3541

EXIST test set:
Number of tweets: 4368
Number of Spanish tweets: 2160

EXIST all posts:
Number of Spanish tweets: 5701
Percentage of hate speech: 50.23680056130503%



###Unification, hashtags and emojis

In [21]:
df = pd.concat([df_hateval, df_exist])

In [22]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_spanish_data.csv', index=False) 

In [13]:
df = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_spanish_data.csv', encoding='utf8', engine='python')

####Emoji code extraction and replacement

In [7]:
emoji_codes = getAllEmojiHexCode(df)

In [8]:
emoji_codes

array([], dtype=float64)

In [ ]:
df = replaceEmojiCodes(df)

####Hashtag and emoji filtering

In [14]:
df, hashtag_dict, emoji_dict = cleanTweets_getDicts(df)

In [10]:
print("HATE SPEECH - PERCENTAGE OF ALL SPANISH POSTS:")
print("WITH HATE SPEECH: " + str((len(df.loc[df['hate speech'] == 1].index)*100)/df.shape[0]))
print("WITHOUT HATE SPEECH: " + str((len(df.loc[df['hate speech'] == 0].index)*100)/df.shape[0]))

HATE SPEECH - PERCENTAGE OF ALL SPANISH POSTS:
WITH HATE SPEECH: 45.54914234615072
WITHOUT HATE SPEECH: 54.45085765384928


#####Removal of all emojis and hashtags

In [15]:
#Remove emojis
df = replace_emojis([], list(emoji_dict.keys()), df)

In [16]:
#Remove hashtags
for idx, row in df.iterrows():
  for k in list(hashtag_dict.keys()):
      row.text = row.text.replace(k, "")
      df['text'][idx] = row.text

In [17]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/removed_spanish_data.csv', index=False)  

#####Filtered emojis and hashtags

In [ ]:
#We divide dictionaries in "important" (elements with at least X number of instances) and "unimportant", for the data with filtered hashtags&emojis
hashtag_dict_important = important_values(hashtag_dict, 10)
hashtag_dict_unimportant = list(filter(lambda i: i not in list(hashtag_dict_important.keys()), list(hashtag_dict.keys())))

emoji_dict_important = important_values(emoji_dict, 10)
emoji_dict_unimportant = list(filter(lambda i: i not in list(emoji_dict_important.keys()), list(emoji_dict.keys())))

In [ ]:
list(hashtag_dict_important.keys())

['#españa',
 '#inmigracion',
 '#inmigrantes',
 '#ceuta',
 '#metoo',
 '#otgala7',
 '#niunamenos',
 '#',
 '#mujeres',
 '#yositecreo',
 '#mgtow',
 '#lamanada',
 '#feminismo',
 '#notallmen']

In [ ]:
list(emoji_dict_important.keys())

[' ',
 '😂',
 '😒',
 '🤣',
 '🏽',
 '😠',
 '💩',
 '🏼',
 '❤',
 '😡',
 '🏻',
 '🔥',
 '👏',
 '😘',
 '🤷',
 '♀',
 '👇',
 '😍',
 '🤔',
 '♂',
 '🤦',
 '👍',
 '🙄',
 '🎶',
 '😭',
 '👑',
 '💛',
 '😏',
 '😢',
 '😈',
 '♥']

In [ ]:
df = replace_emojis()

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/filtered_spanish_data.csv', index=False)  

#####All emojis and hashtags kept

In [ ]:
#Replace emojis with text
df = replace_emojis(list(emoji_dict.keys()), [], df)

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_spanish_data.csv', index=False)  

#ITALIAN DATA UNIFICATION

In [23]:
#AMI
df_ami_train = pd.read_csv('/content/AMI2020_training_raw_anon.csv', encoding='utf8')
df_ami_test = pd.read_csv('/content/AMI2020_test_raw_gold_anon.csv', encoding='utf8')

#IHSC
df_ihsc = pd.read_csv('/content/IHSC_ids.csv', encoding='utf8')

#HaSpeeDe 2
#NOTE: Original .zip has 3 datasets, only train has column names, so we'll just import train set
df_haspee = pd.read_csv('/content/haspeede2_dev_taskAB.csv', encoding='utf8')

###[AMI](https://s3.cbk.cloud.syseleven.net/elg-public/f0fb46717d154339a9929d707bb27f52_u2235_paper161.pdf)

In [24]:
ami_data = "AMI DATASET:\n------------------------------------------\n"

In [25]:
ami_data = ami_data + "AMI train set:\n"
df_ami_train = df_ami_train[['text','misogynous']]
ami_data = ami_data + "Number of Italian tweets: " + str(df_ami_train.shape[0]) + "\n\n"
df_ami_train.rename(columns = {'misogynous':'hate speech'}, inplace = True)

In [26]:
ami_data = ami_data + "AMI test set:\n"
df_ami_test = df_ami_test[['text','misogynous']]
ami_data = ami_data + "Number of Italian tweets: " + str(df_ami_test.shape[0]) + "\n\n"
df_ami_test.rename(columns = {'misogynous':'hate speech'}, inplace = True)

In [27]:
df_ami = pd.concat([df_ami_train, df_ami_test])

In [29]:
ami_data = ami_data + "AMI all posts:\n"
ami_data = ami_data + "Number of Italian tweets: " + str(df_ami.shape[0]) + "\n"
ami_data = ami_data + "Percentage of hate speech: " + str((len(df_ami.loc[df_ami['hate speech'] == 1].index)*100)/df_ami.shape[0]) + "%\n"
print(ami_data)

AMI DATASET:
------------------------------------------
AMI train set:
Number of Italian tweets: 4409

AMI test set:
Number of Italian tweets: 1000

AMI all posts:
Number of Italian tweets: 5409
Percentage of hate speech: 47.08818635607321%



In [30]:
#AMI dataset's URls and Mentions are formatted already as <MENTION_1>, <MENTION_2>, <URL>...
#We eliminate those words here
removed_words_list = []
for idx, row in df_ami.iterrows():
    words = row.text.split()
    for w in words:
      if w.startswith('<'):
        removed_words_list.append(w)
    uniq_elim_words = unique(removed_words_list)
    for el in uniq_elim_words:
      row.text = row.text.replace(el, "")
      df_ami['text'][idx] = row.text
    removed_words_list = []

In [31]:
df_ami

,text,hate speech
0,Fatti trovare te lo do volentieri e ti sbor...,1
1,Tu dovresti ricominciare dai semafori a fare ...,1
2,"Amore,sei presentabile? Xchè così via Skype ti...",1
3,"Salvo poi mandare la culona a Mosca, aummaumm...",1
4,Ti sborro io,1
...,...,...
995,"O si, mi piacerebbe tantissimo, che voglia di...",1
996,Lo so ognuno ha la sua croce se vai su a colo...,1
997,madonna qui partono gli schiaffi,0
998,Che bel culo,1


###[IHSC](https://aclanthology.org/L18-1443.pdf)

####Obtaining tweet text

In [ ]:
df_ihsc = df_ihsc[['tweet_id','hs','tweet_id_full']]
df_ihsc.rename(columns = {'tweet_id':'text'}, inplace = True)
df_ihsc.rename(columns = {'hs':'hate speech'}, inplace = True)

In [ ]:
#Full ids get lost beacuse of formatting
#Let's import the ids separately and then put them in the dataframe
with open('/content/IHSC_ids_ids.txt') as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]

In [ ]:
for idx, row in df_ihsc.iterrows():
  try:
    if api.get_status(lines[idx]):
      tweetFetched = api.get_status(lines[idx])
      df_ihsc['text'][idx] = tweetFetched.text
  except tweepy.TweepError as e:
    # print (e.api_code)
    # print (getExceptionMessage(e.reason))
    df_ihsc['text'][idx] = "TWITTER__ERROR" + getExceptionMessage(e.reason)

df_ihsc.to_csv('/content/drive/MyDrive/TFG/data/ihsc_data.csv', index=False)  

####Rest of preprocessing


In [32]:
df_ihsc = pd.read_csv('/content/drive/MyDrive/TFG/data/ihsc_data.csv', engine="python")

In [33]:
df_ihsc["hate speech"].replace({"yes": int(1), "no": int(0)}, inplace=True)
df_ihsc.round(0)
df_ihsc = df_ihsc[['text','hate speech']]

In [34]:
ihsc_data = "IHSC DATASET:\n------------------------------------------\n"
ihsc_data = ihsc_data + "Before row elimination:\n"
ihsc_data = ihsc_data + "Number of Italian tweets: " + str(df_ihsc.shape[0]) + "\n"
ihsc_data = ihsc_data + "Percentage of hate speech: " + str((len(df_ihsc.loc[df_ihsc['hate speech'] == 1].index)*100)/df_ihsc.shape[0]) + "%\n\n"

In [35]:
for idx, row in df_ihsc.iterrows():
  if "TWITTER__ERROR" in row.text:
    df_ihsc = df_ihsc.drop(labels=idx, axis=0)

In [36]:
ihsc_data = ihsc_data + "After row elimination:\n"
ihsc_data = ihsc_data + "Number of Italian tweets: " + str(df_ihsc.shape[0]) + "\n"
ihsc_data = ihsc_data + "Percentage of hate speech: " + str((len(df_ihsc.loc[df_ihsc['hate speech'] == 1].index)*100)/df_ihsc.shape[0]) + "%\n\n"

In [39]:
df_ihsc

,text,hate speech
0,@Dandy_Dv @M_P_8 @JimmyGhione @virginiaraggi m...,0.0
1,"Studentessa cinese morta a Roma, fermati due g...",0.0
2,"Roma, “tangenti per appalti\nsui campi rom”: c...",0.0
3,"""I rom sono la sola comunità in Italia vittima...",0.0
5,La #Raggi dopo un anno comincia a far qualcosa...,0.0
...,...,...
6931,E dopo #Garland beccatevi questa! #maometto #i...,1.0
6932,@Piovegovernolad Bene.... Poco alla volta i...,1.0
6935,@gintronic4 secondo me è il motore della vita!...,1.0
6938,@gg_frank @EsercitoCrucian sganciamo quel cojo...,0.0


In [38]:
print(ihsc_data)

IHSC DATASET:
------------------------------------------
Before row elimination:
Number of Italian tweets: 6942
Percentage of hate speech: 18.596946125036013%

After row elimination:
Number of Italian tweets: 5205
Percentage of hate speech: 15.830931796349663%




###[HaSpeeDe 2](http://ceur-ws.org/Vol-2765/paper162.pdf)

In [40]:
hsp_data = "HASPEEDE 2 DATASET:\n------------------------------------------\n"

In [41]:
hsp_data = hsp_data + "HASPEEDE 2 train set:\n"
df_haspee = df_haspee.drop(['id', 'stereotype'], axis=1)
hsp_data = hsp_data + "Number of Italian tweets: " + str(df_haspee.shape[0]) + "\n"
df_haspee.rename(columns = {'hs':'hate speech'}, inplace = True)
hsp_data = hsp_data + "Percentage of hate speech: " + str((len(df_haspee.loc[df_haspee['hate speech'] == 1].index)*100)/df_haspee.shape[0]) + "%\n"

In [42]:
#Urls and mentions are already formatted, we delete them here
for idx, row in df_haspee.iterrows():
  row.text = row.text.replace("@user", "")
  row.text = row.text.replace("URL", "")
  df_haspee['text'][idx] = row.text

In [48]:
df_haspee

,text,hate speech
0,"È terrorismo anche questo, per mettere in uno ...",0
1,infatti finché ci hanno guadagnato con i cam...,0
2,"Corriere: Tangenti, Mafia Capitale dimenticata...",0
3,"ad uno ad uno, perché quando i migranti israe...",0
4,Il divertimento del giorno? Trovare i patrioti...,0
...,...,...
6832,Gli stati nazionali devono essere pronti a rin...,0
6833,Il ministro dell'interno della Germania #Horst...,0
6834,#Salvini: In Italia troppi si sono montati la ...,0
6835,Chi giubila in buona fede non ha capito nien...,0


In [44]:
print(hsp_data)

HASPEEDE 2 DATASET:
------------------------------------------
HASPEEDE 2 train set:
Number of Italian tweets: 6837
Percentage of hate speech: 40.4563405002194%



###Unification, hashtags and emojis

In [49]:
df = pd.concat([df_ami, df_ihsc, df_haspee])

In [50]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_italian_data.csv', index=False) 

In [18]:
df = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_italian_data.csv', encoding='utf8', engine='python')

####Emoji code extraction and replacement

In [19]:
emoji_codes = getAllEmojiHexCode(df)

In [20]:
emoji_codes

array(['&#x1f339;', '&#x1f346;', '&#x1f399;', '&#x1f3c1;', '&#x1f42e;',
       '&#x1f433;', '&#x1f437;', '&#x1f441;', '&#x1f444;', '&#x1f445;',
       '&#x1f447;', '&#x1f44d;', '&#x1f47f;', '&#x1f48b;', '&#x1f493;',
       '&#x1f496;', '&#x1f4a6;', '&#x1f4a9;', '&#x1f4aa;', '&#x1f4af;',
       '&#x1f50b;', '&#x1f51d;', '&#x1f600;', '&#x1f601;', '&#x1f602;',
       '&#x1f605;', '&#x1f606;', '&#x1f608;', '&#x1f609;', '&#x1f60b;',
       '&#x1f60d;', '&#x1f611;', '&#x1f614;', '&#x1f618;', '&#x1f61a;',
       '&#x1f61c;', '&#x1f621;', '&#x1f62d;', '&#x1f630;', '&#x1f643;',
       '&#x1f645;', '&#x1f64f;', '&#x1f914;', '&#x1f921;', '&#x1f923;',
       '&#x1f924;', '&#x1f926;', '&#x1f952;', '&#x25b6;', '&#x2615;',
       '&#x26a1;', '&#x270a;', '&#x270c;', '&#x2753;', '&#x2764;',
       '&#x27a1;'], dtype='<U9')

In [ ]:
df = replaceEmojiCodes(df)

####Hashtag and emoji filtering

In [ ]:
df, hashtag_dict, emoji_dict = cleanTweets_getDicts(df)

In [ ]:
hashtag_dict_important = important_values(hashtag_dict, 30)
emoji_dict_important = important_values(emoji_dict, 30)

In [ ]:
hashtag_dict_important

{'#': 52,
 '#amici17': 32,
 '#berlino': 31,
 '#dallavostraparte': 101,
 '#immigrati': 59,
 '#immigrazione': 59,
 '#isis': 41,
 '#islam': 42,
 '#italia': 107,
 '#lidl': 35,
 '#migranti': 247,
 '#milano': 143,
 '#news': 168,
 '#notizie': 41,
 '#piazzapulita': 36,
 '#profughi': 35,
 '#quintacolonna': 47,
 '#raggi': 32,
 '#rom': 210,
 '#roma': 256,
 '#salvini': 71,
 '#terrorismo': 114}

In [ ]:
emoji_dict_important

{' ': 1297,
 '❤': 54,
 '👅': 80,
 '👏': 59,
 '💦': 132,
 '😂': 76,
 '😋': 62,
 '😍': 60,
 '😡': 45,
 '🤣': 64}

In [21]:
print("HATE SPEECH - PERCENTAGE OF ALL ITALIAN POSTS:")
print("WITH HATE SPEECH: " + str((len(df.loc[df['hate speech'] == 1].index)*100)/df.shape[0]))
print("WITHOUT HATE SPEECH: " + str((len(df.loc[df['hate speech'] == 0].index)*100)/df.shape[0]))

HATE SPEECH - PERCENTAGE OF ALL ITALIAN POSTS:
WITH HATE SPEECH: 35.16703913815827
WITHOUT HATE SPEECH: 64.75273623288064


#####Removal of all emojis and hashtags

In [ ]:
#Remove emojis
df = replace_emojis([], list(emoji_dict.keys()), df)

In [ ]:
#Remove hashtags
for idx, row in df.iterrows():
  for k in list(hashtag_dict.keys()):
      row.text = row.text.replace(k, "")
      df['text'][idx] = row.text

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/removed_italian_data.csv', index=False)  

#####Filtered emojis and hashtags

In [ ]:
df = replace_emojis()

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/filtered_italian_data.csv', index=False)  

#####All emojis and hashtags kept

In [ ]:
#Replace emojis with text
df = replace_emojis(list(emoji_dict.keys()), [], df)

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_italian_data.csv', index=False)  

#PORTUGUESE DATA UNIFICATION

In [51]:
#H-L PHS
df_hlphs = pd.read_csv('/content/2019-05-28_portuguese_hate_speech_hierarchical_classification.csv', encoding='utf8')

#ToLD-BR
df_told = pd.read_csv('/content/ToLD-BR.csv', encoding='utf8')

#OffComBR -> OFFCOMBR-3 only contains instances for which the class has been agreed by all three annotators (we'll use that dataset)
df_ocombr = pd.read_csv('/content/csv_result-OffComBR3.csv', encoding='utf8')

###[Hierarchically-Labeled Portuguese Hate Speech](https://aclanthology.org/W19-3510.pdf)

In [59]:
hlphs_data = "HIERARCHICALLY-LABELED PORTUGUESE HATE SPEECH DATASET:\n------------------------------------------\n"

In [60]:
#List of columns we want to mantain (deleted ones are non-socially oppressed targets: gamers, men, white people, etc.)
columns_to_mantain = ["text",
 "Sexism",
 "Racism",
 "Homophobia",
 "Origin",
 "Religion",
 "Health",
 "OtherLifestyle",
 "Aborting.women",
 "Argentines",
 "Asians",
 "Autists",
 "Black.Women",
 "Blond.women",
 "Brazilians.women",
 "Chinese",
 "Egyptians",
 "Fat.people",
 "Football.players.women",
 "Gamers",
 "Homeless",
 "Homeless.women",
 "Indigenous",
 "Iranians",
 "Japaneses",
 "Jews",
 "Latins",
 "Mexicans",
 "Muslims.women",
 "Nordestines",
 "Poor.people",
 "Rural.people",
 "Sertanejos",
 "Old.women",
 "Ugly.people",
 "Venezuelans",
 "Angolans",
 "Black.people",
 "Disabled.people",
 "Fat.women",
 "Feminists",
 "Gays",
 "Immigrants",
 "Islamists",
 "Lesbians",
 "Muslims",
 "Refugees",
 "Trans.women",
 "Travestis",
 "Women",
 "Thin.women",
 "Bissexuals",
 "Transexuals",
 "Ugly.women",
 "Arabic",
 "Africans",
 "South.Americans",
 "Brazilians",
 "Migrants",
 "Homossexuals"]

In [61]:
columns_to_delete = ["Body",
"Ideology",
"Agnostic",	
"Criminals",	
"Jornalists",	
"Left.wing.ideology",
"Men.Feminists",
"Old.people",
"Polyamorous",
"Russians",
"Street.artist",	
"Ucranians",
"Vegetarians",	
"White.people",	
"Young.people",
"Men",	
"East.europeans",	
"Thin.people",
"Ageing"]

In [62]:
hier_class_dict = { 'Sexism': {'Women': {'Lesbians': 'Lesbians',
                                         'Fat.women': 'Fat.women',
                                         'Ugly.women': 'Ugly.women',
                                         'Trans.women': 'Trans.women',
                                         'Black.Women': 'Black.Women',
                                         'Travestis': 'Travestis',
                                         'Aborting.women': 'Aborting.women',
                                         'Blond.women': 'Blond.women',
                                         'Thin.women': 'Thin.women',
                                         'Brazilians.women':'Brazilians.women',
                                         'Football.players.women': 'Football.players.women',
                                         'Homeless.women': 'Homeless.women',
                                         'Muslims.women': 'Muslims.women',
                                         'Old.women': 'Old.women'},
                               'Men': {'Men.Feminists': 'Men.Feminists'},
                               'Transexuals': {'Trans.women': 'Trans.women'},
                               'Feminists': {'Men.Feminists': 'Men.Feminists'}},
                'Homophobia': {'Homossexuals': {'Gays': 'Gays',
                                                'Lesbians': 'Lesbians'},
                               'Bissexuals': 'Bissexuals'},
                'Body': {'Fat.people': {'Fat.women': 'Fat.women'},
                         'Ugly.people': {'Ugly.women': 'Ugly.women'},
                         'Thin.people': {'Thin.women': 'Thin.women'},
                         'Blond.women': 'Blond.women'},
                'Racism': {'Latins': {'Argentines': 'Argentines',
                                      'Mexicans': 'Mexicans',
                                      'Venezuelans': 'Venezuelans'},
                           'Black.people': {'Black.Women': 'Black.Women'},
                           'Asians': {'Chinese': 'Chinese',
                                      'Japaneses': 'Japaneses'},
                           'White.people': 'White.people',
                           'Indigenous': 'Indigenous'},
                'Ideology': {'Feminists': {'Men.Feminists': 'Men.Feminists'},
                             'Agnostic': 'Agnostic',
                             'Left.wing.ideology': 'Left.wing.ideology'},
                'Migrants': {'Refugees': 'Refugees',
                             'Immigrants': 'Immigrants'},
                'Origin': {'Latins': {'Argentines': 'Argentines',
                                      'Mexicans': 'Mexicans',
                                      'Venezuelans': 'Venezuelans'},
                           'Rural.people': {'Sertanejos': 'Sertanejos',
                                            'Nordestines': 'Nordestines'},
                           'Asians': {'Chinese': 'Chinese',
                                      'Japaneses': 'Japaneses'},
                           'South.Americans': {'Brazilians': {'Sertanejos': 'Sertanejos',
                                                              'Nordestines': 'Nordestines'},
                                               'Brazilians.women':'Brazilians.women'},
                           'Africans': {'Angolans': 'Angolans'},
                           'Arabic': {'Egyptians': 'Egyptians',
                                      'Iranians': 'Iranians'},
                           'East.europeans': {'Russians': 'Russians',
                                              'Ucranians': 'Ucranians'}},
                'Religion': {'Islamists': 'Islamists',
                             'Muslims': {'Muslims.women': 'Muslims.women'},
                             'Jews': 'Jews'},
                'OtherLifestyle': {'Criminals': 'Criminals',
                                   'Homeless': {'Homeless.women': 'Homeless.women'},
                                   'Jornalists': 'Jornalists',
                                   'Vegetarians': 'Vegetarians',
                                   'Football.players.women': 'Football.players.women',
                                   'Gamers': 'Gamers',
                                   'Polyamorous': 'Polyamorous',
                                   'Poor.people': 'Poor.people',
                                   'Street.artist': 'Street.artist'},
                'Health': {'Disabled.people': 'Disabled.people',
                           'Autists': 'Autists'},
                'Ageing': {'Old.people': {'Old.women': 'Old.women'},
                           'Young.people': 'Young.people'}}

In [63]:
#Changing useless column values + their parent columns' column value
for idx, row in df_hlphs.iterrows():
  for col_del in columns_to_delete:
    if str(row[col_del]) == "1": 
      df_hlphs[col_del][idx] = 0
      for class_del in list(filter(lambda a: a != col_del, find_parent_classes(hier_class_dict, col_del))):
        df_hlphs[class_del][idx] = 0

In [64]:
#Do the same process but for the columns we DO want to mantain (we could do without this, but it makes sure the dataframe structure is coherent)
for idx, row in df_hlphs.iterrows():
  for col_man in columns_to_mantain:
    if str(row[col_man]) == "1": 
      df_hlphs[col_man][idx] = 1
      for class_man in list(filter(lambda a: a != col_man, find_parent_classes(hier_class_dict, col_man))):
        df_hlphs[class_man][idx] = 1

In [65]:
df_hlphs = df_hlphs[columns_to_mantain]
hlphs_data = hlphs_data + "Number of Portuguese tweets: " + str(df_hlphs.shape[0]) + "\n"
df_hlphs["hate speech"] = 0
#"hate speech column is and "or" of all columns, if 1 or more columns have a 1, it counts as hate speech"
for col in df_hlphs.columns[1:]:
  df_hlphs["hate speech"] = df_hlphs["hate speech"] + df_hlphs[col]
df_hlphs = df_hlphs[["text", "hate speech"]]
df_hlphs['hate speech'] = df_hlphs['hate speech'].replace([df_hlphs['hate speech'].unique()[1:]],1)
hlphs_data = hlphs_data + "Percentage of hate speech: " + str((len(df_hlphs.loc[df_hlphs['hate speech'] == 1].index)*100)/df_hlphs.shape[0]) + "%\n"

In [66]:
df_hlphs

,text,hate speech
0,"""não come mel, morde marimbondo""",0
1,"não tem pinto, tem orgulho !",0
2,Não vê essa merda de Crepúsculo! Pra isso temo...,0
3,"não da tapa na bundinha, da cotovelada nas cos...",0
4,o diminutivo INHO não acompanha a trajetória d...,0
...,...,...
5663,Na minha sala só tem viado e sapatão e a cois...,1
5664,PARABENS SAPATÃO SDDS @attomiter https://t.co/...,1
5665,RT @toquedeveludo: Agora um poema:\nEu sou sap...,1
5666,O mundo das sapatao é mais ligado do que eu im...,1


In [67]:
print(hlphs_data)

HIERARCHICALLY-LABELED PORTUGUESE HATE SPEECH DATASET:
------------------------------------------
Number of Portuguese tweets: 5668
Percentage of hate speech: 19.848270995059988%



###[ToLD-BR](https://arxiv.org/pdf/2010.04543.pdf)

"Each tweet was annotated by three different annotators."; therefore a tweet need a 2 or 3 in any of the hate speech columns: homophobia, racism, misogyny, xenphobia


In [56]:
told_data = "TOLD-BR DATASET:\n------------------------------------------\n"

In [57]:
#Obscene and insulting tweets don't count as hate speech, we drop these columns
df_told = df_told.drop(['obscene', 'insult'], axis=1)
told_data = told_data + "Number of Portuguese tweets: " + str(df_told.shape[0]) + "\n"
df_told["hate speech"] = 0
for col in df_told.columns[1:]:
  #Only if 2-3 annotators (the majority) judge a tweet as hate speech, we count is as hate speech
  df_told[col] = df_told[col].replace([1],0)
  df_told[col] = df_told[col].replace([2, 3],1)
#"hate speech column is and "or" of all columns, if 1 or more columns have a 1, it counts as hate speech"
for col in df_told.columns[1:5]:
  df_told["hate speech"] = df_told["hate speech"] + df_told[col]
df_told['hate speech'] = df_told['hate speech'].replace([df_told['hate speech'].unique()[1:]],1)
told_data = told_data + "Percentage of hate speech: " + str((len(df_told.loc[df_told['hate speech'] == 1].index)*100)/df_told.shape[0]) + "%\n"
df_told = df_told[["text", "hate speech"]]

In [68]:
df_told

,text,hate speech
0,Meu nivel de amizade com isis é ela ter meu in...,0.0
1,"rt @user @user o cara adultera dados, que fora...",0.0
2,@user @user @user o cara só é simplesmente o m...,0.0
3,eu to chorando vei vsf e eu nem staneio izone ...,0.0
4,Eleitor do Bolsonaro é tão ignorante q não per...,0.0
...,...,...
20995,@user faz favor vai carai,0.0
20996,só queria conhecer alguém que não conhece o he...,0.0
20997,"vcs militam na hora errada em cima de memes, p...",0.0
20998,@user porra any eu tava c dor de cabeca e fui ...,0.0


In [58]:
print(told_data)

TOLD-BR DATASET:
------------------------------------------
Number of Portuguese tweets: 21000
Percentage of hate speech: 1.7904761904761906%



###[OffComBR](http://www.each.usp.br/digiampietri/BraSNAM/2017/p04.pdf)
The source of the data was the news site g1.globo.com. This is the most accessed
news site in Brazil12 and, as a result, it has many comments. Although the comments on
this site go through moderation, we found a considerable number of offensive contents.


In [52]:
ocombr_data = "OFFCOMBR DATASET:\n------------------------------------------\n"

In [53]:
#The only necessary work here is to replace column values and names
df_ocombr = df_ocombr.drop(['id'], axis=1)
ocombr_data = ocombr_data + "Number of Portuguese tweets: " + str(df_ocombr.shape[0]) + "\n"
df_ocombr.rename(columns = {'@@class':'hate speech'}, inplace = True)
df_ocombr.rename(columns = {'document':'text'}, inplace = True)
df_ocombr = df_ocombr.reindex(columns=['text','hate speech'])
df_ocombr["hate speech"].replace({"yes": int(1), "no": int(0)}, inplace=True)
ocombr_data = ocombr_data + "Percentage of hate speech: " + str((len(df_ocombr.loc[df_ocombr['hate speech'] == 1].index)*100)/df_ocombr.shape[0]) + "%\n"

In [54]:
df_ocombr

,text,hate speech
0,Votaram no PEZAO Agora tomem no CZAO,1
1,cuidado com a poupanca pessoal Lembram o que a...,0
2,Sabe o que eu acho engracado os nossos governa...,0
3,Podiam retirar dos lucros dos bancos,0
4,CADE O GALVAO PRA NARRAR AGORA FALIIIIUUUUUU...,0
...,...,...
1028,Cruz so tem agilidade mesmo poder de nocaute z...,0
1029,Meus caros amigos enigmaticosNao deveriam comp...,0
1030,Ele chamava pra atras da escola e sentava nos ...,1
1031,Jhalim Rabei ate fiquei assustado comecei a le...,0


In [55]:
print(ocombr_data)

OFFCOMBR DATASET:
------------------------------------------
Number of Portuguese tweets: 1033
Percentage of hate speech: 19.554695062923525%



###Unification, hashtags and emojis

In [69]:
df = pd.concat([df_hlphs, df_told, df_ocombr])

In [70]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_portuguese_data.csv', index=False)

In [22]:
df = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_portuguese_data.csv', encoding='utf8', engine='python')

####Emoji code extraction and replacement

In [23]:
emoji_codes = getAllEmojiHexCode(df)

In [24]:
emoji_codes

array([], dtype=float64)

In [ ]:
df = replaceEmojiCodes(df)

####Hashtag and emoji filtering

In [25]:
df, hashtag_dict, emoji_dict = cleanTweets_getDicts(df)

In [26]:
hashtag_dict_important = important_values(hashtag_dict, 10)
emoji_dict_important = important_values(emoji_dict, 10)

In [27]:
hashtag_dict_important

{'#diadamulher': 36,
 '#joaquinresponde': 29,
 '#luanetes': 18,
 '#luansantana': 18,
 '#luansantanashow': 18,
 '#marinaruybarbosa': 20,
 '#masterchefbr': 24,
 '#mpn': 131,
 '#mulherdeverdade': 30,
 '#nasruascontrastf': 27,
 '#nowunited': 86,
 '#oscars': 15,
 '#paz': 30,
 '#pnr': 85,
 '#portugal': 50,
 '#renovarportugal': 48,
 '#sermulhere': 17,
 '#sm_보이그룹론칭_취소해': 11,
 '#stfvergonhanacional': 135,
 '#stfvergonhanascional': 11,
 '#uniters': 65,
 '#voleinaglobo': 79,
 '#voleinosportv': 18}

In [28]:
emoji_dict_important

{' ': 6194,
 '☺': 13,
 '♀': 84,
 '♂': 61,
 '♥': 34,
 '⚠': 26,
 '⚪': 16,
 '⚫': 12,
 '⚽': 13,
 '✅': 20,
 '✌': 16,
 '✨': 35,
 '❤': 326,
 '🌠': 11,
 '🌸': 17,
 '🍆': 24,
 '🍑': 11,
 '🍻': 12,
 '🎂': 11,
 '🎉': 12,
 '🎵': 13,
 '🎶': 39,
 '🏻': 169,
 '🏼': 52,
 '🏽': 41,
 '🏾': 67,
 '👉': 23,
 '👊': 24,
 '👌': 20,
 '👍': 78,
 '👎': 12,
 '👏': 82,
 '👺': 12,
 '💃': 15,
 '💓': 13,
 '💔': 40,
 '💕': 44,
 '💖': 42,
 '💗': 18,
 '💘': 18,
 '💙': 29,
 '💚': 29,
 '💛': 24,
 '💜': 74,
 '💞': 21,
 '💥': 12,
 '💦': 27,
 '💪': 19,
 '🔄': 26,
 '🔝': 16,
 '🔥': 79,
 '🔴': 13,
 '🖕': 11,
 '🖤': 14,
 '🗣': 12,
 '😁': 16,
 '😂': 833,
 '😅': 36,
 '😆': 16,
 '😈': 86,
 '😉': 29,
 '😊': 11,
 '😋': 33,
 '😌': 12,
 '😍': 216,
 '😎': 35,
 '😏': 21,
 '😒': 25,
 '😓': 13,
 '😔': 107,
 '😘': 29,
 '😚': 11,
 '😜': 14,
 '😝': 23,
 '😞': 13,
 '😠': 44,
 '😡': 139,
 '😢': 13,
 '😣': 16,
 '😤': 36,
 '😩': 33,
 '😪': 17,
 '😬': 21,
 '😭': 227,
 '😰': 11,
 '😱': 21,
 '😴': 12,
 '😹': 13,
 '😻': 14,
 '🙄': 59,
 '🙌': 27,
 '🙏': 82,
 '🚨': 11,
 '🤑': 20,
 '🤔': 50,
 '🤙': 13,
 '🤠': 11,
 '🤢': 18,
 '🤣': 242,


In [29]:
print("HATE SPEECH - PERCENTAGE OF ALL PORTUGUESE POSTS:")
print("WITH HATE SPEECH: " + str((len(df.loc[df['hate speech'] == 1].index)*100)/df.shape[0]))
print("WITHOUT HATE SPEECH: " + str((len(df.loc[df['hate speech'] == 0].index)*100)/df.shape[0]))

HATE SPEECH - PERCENTAGE OF ALL PORTUGUESE POSTS:
WITH HATE SPEECH: 6.147792498465759
WITHOUT HATE SPEECH: 93.85220750153424


#####Removal of all emojis and hashtags

In [ ]:
#Remove emojis
df = replace_emojis([], list(emoji_dict.keys()), df)

In [ ]:
#Remove hashtags
for idx, row in df.iterrows():
  for k in list(hashtag_dict.keys()):
      row.text = row.text.replace(k, "")
      df['text'][idx] = row.text

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/removed_portuguese_data.csv', index=False)  

#####Filtered emojis and hashtags

In [ ]:
df = replace_emojis()

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/filtered_portuguese_data.csv', index=False)  

#####All emojis and hashtags kept

In [ ]:
#Replace emojis with text
df = replace_emojis(list(emoji_dict.keys()), [], df)

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_portuguese_data.csv', index=False)  

#ENGLISH DATA UNIFICATION

In [71]:
#hateval
df_hateval_train = pd.read_csv('/content/hateval2019_en_train.csv', encoding='utf8')
df_hateval_test = pd.read_csv('/content/hateval2019_en_test.csv', encoding='utf8')
df_hateval_val = pd.read_csv('/content/hateval2019_en_dev.csv', encoding='utf8')

#EXIST
df_exist_train = pd.read_csv('/content/EXIST2021_training.csv', encoding='utf8', engine='python')
df_exist_test = pd.read_csv('/content/EXIST2021_test_labeled.csv', encoding='utf8', engine='python')

#Automated Hate Speech Detection
df_aut_hate = pd.read_csv('/content/labeled_data_dav_en.csv', encoding='utf8')

#Hateful Symbols or Hateful People?
df_hatesym = pd.read_csv('/content/NAACL_SRW_2016.csv', encoding='utf8', header= None)

#Are You a Racist or Am I Seeing Things?
df_ruracist = pd.read_csv('/content/NLP+CSS_2016.csv', encoding='utf8', sep='\t')

#Hate speech dataset from a white supremacist forum 
df_whitesup = pd.read_csv('/content/annotations_metadata_white_sup.csv', encoding='utf8')

#The Gab Hate Corpus
df_ghc_train = pd.read_csv('/content/ghc_train.csv', encoding='utf8')
df_ghc_test = pd.read_csv('/content/ghc_test.csv', encoding='utf8')

###[HATEVAL](https://aclanthology.org/S19-2007.pdf)

In [72]:
hateval_data = "HATEVAL DATASET:\n------------------------------------------\n"

In [73]:
hateval_data = hateval_data + "hatEval train set:\n"
#We won't use the columns that determined aggresiveness or target (group or individual) -> case is the same in all 3 subsets
df_hateval_train = df_hateval_train.drop(['TR', 'AG'], axis=1)
hateval_data = hateval_data + "Number of English tweets: " + str(df_hateval_train.shape[0]) + "\n\n"
df_hateval_train = df_hateval_train[['text','HS']]
df_hateval_train.rename(columns = {'HS':'hate speech'}, inplace = True)

In [74]:
hateval_data = hateval_data + "hatEval test set:\n"
hateval_data = hateval_data + "Number of English tweets: " + str(df_hateval_test.shape[0]) + "\n\n"
df_hateval_test = df_hateval_test[['text','HS']]
df_hateval_test.rename(columns = {'HS':'hate speech'}, inplace = True)

In [75]:
hateval_data = hateval_data + "hatEval validation set:\n"
df_hateval_val = df_hateval_val.drop(['TR', 'AG'], axis=1)
hateval_data = hateval_data + "Number of English tweets: " + str(df_hateval_val.shape[0]) + "\n\n"
df_hateval_val = df_hateval_val[['text','HS']]
df_hateval_val.rename(columns = {'HS':'hate speech'}, inplace = True)

In [76]:
df_hateval = pd.concat([df_hateval_train, df_hateval_val, df_hateval_test])

In [77]:
df_hateval

,text,hate speech
0,"Hurray, saving us $$$ in so many ways @potus @...",1
1,Why would young fighting age men be the vast m...,1
2,@KamalaHarris Illegals Dump their Kids at the ...,1
3,NY Times: 'Nearly All White' States Pose 'an A...,0
4,Orban in Brussels: European leaders are ignori...,0
...,...,...
2995,you can never take a L off a real bitch😩 im ho...,1
2996,@Brian_202 likes to call me a cunt & a bitch b...,1
2997,@kusha1a @Camio_the_wise @shoe0nhead 1. Never ...,1
2998,If i see and know you a hoe why would i hit yo...,1


In [78]:
hateval_data = hateval_data + "hatEval all posts:\n"
hateval_data = hateval_data + "Number of English tweets: " + str(df_hateval.shape[0]) + "\n"
hateval_data = hateval_data + "Percentage of hate speech: " + str((len(df_hateval.loc[df_hateval['hate speech'] == 1].index)*100)/df_hateval.shape[0]) + "%\n"
print(hateval_data)

HATEVAL DATASET:
------------------------------------------
hatEval train set:
Number of English tweets: 9000

hatEval test set:
Number of English tweets: 3000

hatEval validation set:
Number of English tweets: 1000

hatEval all posts:
Number of English tweets: 13000
Percentage of hate speech: 42.07692307692308%



###[EXIST](http://journal.sepln.org/sepln/ojs/ojs/index.php/pln/article/viewFile/6389/3809)

In [79]:
exist_data = "EXIST DATASET:\n------------------------------------------\n"

In [80]:
exist_data = exist_data + "EXIST train set:\n"
exist_data = exist_data + "Number of tweets: " + str(df_exist_train.shape[0]) + "\n"
#Choose corresponding language
df_exist_train = df_exist_train[df_exist_train['language']=="en"]
exist_data = exist_data + "Number of English tweets: " + str(df_exist_train.shape[0]) + "\n\n"
#We just use the first task (binary value: sexist or non-sexist)
df_exist_train['task1'] = df_exist_train['task1'].replace(['non-sexist'],0)
df_exist_train['task1'] = df_exist_train['task1'].replace(['sexist'],1)
df_exist_train = df_exist_train[['text','task1']]
df_exist_train.rename(columns = {'task1':'hate speech'}, inplace = True)

In [81]:
exist_data = exist_data + "EXIST test set:\n"
exist_data = exist_data + "Number of tweets: " + str(df_exist_test.shape[0]) + "\n"
#Choose corresponding language
df_exist_test = df_exist_test[df_exist_test['language']=="en"]
exist_data = exist_data + "Number of English tweets: " + str(df_exist_test.shape[0]) + "\n\n"
#We just use the first task (binary value: sexist or non-sexist)
df_exist_test['task1'] = df_exist_test['task1'].replace(['non-sexist'],0)
df_exist_test['task1'] = df_exist_test['task1'].replace(['sexist'],1)
df_exist_test = df_exist_test[['text','task1']]
df_exist_test.rename(columns = {'task1':'hate speech'}, inplace = True)

In [82]:
df_exist = pd.concat([df_exist_train, df_exist_test])

In [83]:
df_exist

,text,hate speech
0,"She calls herself ""anti-feminazi"" how about sh...",1
1,"Now, back to these women, the brave and the be...",0
2,"@CurvyBandida @Xalynne_B Wow, your skirt is ve...",1
3,@AurelieGuiboud Incredible! Beautiful!But I l...,0
4,i find it extremely hard to believe that kelly...,0
...,...,...
2203,@Hazel08539106 @hatterme123 @lilhazelnuttt You...,0
2204,"how are you, a grown ass woman that’s literall...",0
2205,Another gold digger looking to make money off ...,1
2206,We have another first!! So glad to see women d...,0


In [84]:
exist_data = exist_data + "EXIST all posts:\n"
exist_data = exist_data + "Number of English tweets: " + str(df_exist.shape[0]) + "\n"
exist_data = exist_data + "Percentage of hate speech: " + str((len(df_exist.loc[df_exist['hate speech'] == 1].index)*100)/df_exist.shape[0]) + "%\n"
print(exist_data)

EXIST DATASET:
------------------------------------------
EXIST train set:
Number of tweets: 6977
Number of English tweets: 3436

EXIST test set:
Number of tweets: 4368
Number of English tweets: 2208

EXIST all posts:
Number of English tweets: 5644
Percentage of hate speech: 49.50389794472006%



###[Automated Hate Speech Detection](https://ojs.aaai.org/index.php/ICWSM/article/view/14955/14805)


In [85]:
aut_hate_data = "AUTOMATED HATE SPEECH DETECTION DATASET:\n------------------------------------------\n"

In [86]:
aut_hate_data = aut_hate_data + "Number of English tweets: " + str(df_aut_hate.shape[0]) + "\n"
#Replacing class names
#Original classes: 0 - hate speech; 1 - offensive  language; 2 - neither"
#We transform that into 1 (hate speech) and 0 (not hate speech)
df_aut_hate['class'] = df_aut_hate['class'].replace([1],2)
df_aut_hate['class'] = df_aut_hate['class'].replace([0],1)
df_aut_hate['class'] = df_aut_hate['class'].replace([2],0)
df_aut_hate = df_aut_hate[['class','tweet']]
df_aut_hate.rename(columns = {'class':'hate speech'}, inplace = True)
df_aut_hate.rename(columns = {'tweet':'text'}, inplace = True)
aut_hate_data = aut_hate_data + "Percentage of hate speech: " + str((len(df_aut_hate.loc[df_aut_hate['hate speech'] == 1].index)*100)/df_aut_hate.shape[0]) + "%\n"

In [87]:
df_aut_hate

,hate speech,text
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,0,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,0,"you've gone and broke the wrong heart baby, an..."
24780,0,young buck wanna eat!!.. dat nigguh like I ain...
24781,0,youu got wild bitches tellin you lies


In [88]:
print(aut_hate_data)

AUTOMATED HATE SPEECH DETECTION DATASET:
------------------------------------------
Number of English tweets: 24783
Percentage of hate speech: 5.770084332001775%



###[Hateful Symbols or Hateful People?](https://aclanthology.org/N16-2013.pdf)

####Obtaining tweet text

In [24]:
df_hatesym.rename(columns = {0:'text'}, inplace = True)
df_hatesym.rename(columns = {1:'hate speech'}, inplace = True)
df_hatesym['hate speech'] = df_hatesym['hate speech'].replace(['none'],0)
df_hatesym['hate speech'] = df_hatesym['hate speech'].replace(['sexism', 'racism'],1)

In [125]:
for idx, row in df_hatesym.iterrows():
  #This condition is for when execution is stopped and later on resumed
  if not any(c.isalpha() for c in str(df_hatesym['text'][idx])):
    try:
      if api.get_status(df_hatesym['text'][idx]):
        tweetFetched = api.get_status(df_hatesym['text'][idx])
        df_hatesym['text'][idx] = tweetFetched.text
    except tweepy.TweepError as e:
      # print (e.api_code)
      # print (getExceptionMessage(e.reason))
      df_hatesym['text'][idx] = "TWITTER__ERROR" + getExceptionMessage(e.reason)
    #We save the dataframe every line to not lose any progress (in case this cell would stop after 6+ hours!)
    df_hatesym.to_csv('/content/drive/MyDrive/TFG/data/hatesym_data.csv', index=False)  


####Rest of preprocessing


In [89]:
df_hatesym = pd.read_csv('/content/drive/MyDrive/TFG/data/hatesym_data.csv', engine="python")
#This dataframe contains some rows corrected (check out next dataset's process to see why)
df_hatesym_correct = pd.read_csv('/content/drive/MyDrive/TFG/data/hatesym_data_correctclass.csv', engine="python")

In [90]:
hatesym_data = "HATEFUL SYMBOLS OR HATEFUL PEOPLE? DATASET:\n------------------------------------------\n"
hatesym_data = hatesym_data + "Before row elimination:\n"
hatesym_data = hatesym_data + "Number of English tweets: " + str(df_hatesym.shape[0]) + "\n"
hatesym_data = hatesym_data + "Percentage of hate speech: " + str((len(df_hatesym.loc[df_hatesym['hate speech'] == 1].index)*100)/df_hatesym.shape[0]) + "%\n\n"

In [91]:
for idx, row in df_hatesym.iterrows():
  if "TWITTER__ERROR" in row.text:
    df_hatesym = df_hatesym.drop(labels=idx, axis=0)

In [92]:
hatesym_data = hatesym_data + "After row elimination:\n"
hatesym_data = hatesym_data + "Number of English tweets: " + str(df_hatesym.shape[0]) + "\n"
hatesym_data = hatesym_data + "Percentage of hate speech: " + str((len(df_hatesym.loc[df_hatesym['hate speech'] == 1].index)*100)/df_hatesym.shape[0]) + "%\n\n"

In [93]:
#In Waseem's Are you Racist or Am I seeing Things? dataset, some of the tweets overlap the ones used here, and I manually checked tweets where classification in both datasets is different
hatesym_data = hatesym_data + "After small row correction:\n"
hatesym_data = hatesym_data + "Number of English tweets: " + str(df_hatesym_correct.shape[0]) + "\n"
hatesym_data = hatesym_data + "Percentage of hate speech: " + str((len(df_hatesym_correct.loc[df_hatesym_correct['hate speech'] == 1].index)*100)/df_hatesym_correct.shape[0]) + "%\n\n"

In [94]:
for idx, row in df_hatesym_correct.iterrows():
  if "TWITTER__ERROR" in row.text:
    df_hatesym_correct = df_hatesym_correct.drop(labels=idx, axis=0)

In [95]:
hatesym_data = hatesym_data + "After small row correction and row elimination:\n"
hatesym_data = hatesym_data + "Number of English tweets: " + str(df_hatesym_correct.shape[0]) + "\n"
hatesym_data = hatesym_data + "Percentage of hate speech: " + str((len(df_hatesym_correct.loc[df_hatesym_correct['hate speech'] == 1].index)*100)/df_hatesym_correct.shape[0]) + "%\n\n"

In [96]:
df_hatesym = df_hatesym_correct

In [97]:
df_hatesym

,text,hate speech
0,So Drasko just said he was impressed the girls...,1.0
1,Drasko they didn't cook half a bird you idiot ...,1.0
2,Hopefully someone cooks Drasko in the next ep ...,1.0
3,of course you were born in serbia...you're as ...,1.0
4,These girls are the equivalent of the irritati...,1.0
...,...,...
16825,@ProoN ❤️,0.0
16826,RT @Transition: @freebsdgirl just so I’m clear...,0.0
16828,RT @Leonard_Delaney: @freebsdgirl I know! Holy...,0.0
16829,"FLOSS Weekly, open source projects, and paying...",0.0


In [98]:
print(hatesym_data)

HATEFUL SYMBOLS OR HATEFUL PEOPLE? DATASET:
------------------------------------------
Before row elimination:
Number of English tweets: 16909
Percentage of hate speech: 31.628127032941038%

After row elimination:
Number of English tweets: 10242
Percentage of hate speech: 26.303456356180433%

After small row correction:
Number of English tweets: 16909
Percentage of hate speech: 31.59264297119877%

After small row correction and row elimination:
Number of English tweets: 10242
Percentage of hate speech: 26.24487404803749%




###[Are You a Racist or Am I Seeing Things?](https://aclanthology.org/W16-5618.pdf)

"Our data set
extends the Waseem and Hovy (2016) data set by
4, 033 tweets."
"Our data set is obtained by sampling tweets from the
130k tweets extracted by Waseem and Hovy (2016)."

####Obtaining tweet text

In [113]:
df_hatesym_ids = pd.read_csv('/content/NAACL_SRW_2016.csv', encoding='utf8', header= None)

In [114]:
df_hatesym_ids.rename(columns = {0:'id'}, inplace = True)
df_hatesym_ids.rename(columns = {1:'hs'}, inplace = True)
df_hatesym_ids['hs'] = df_hatesym_ids['hs'].replace(['none'],0)
df_hatesym_ids['hs'] = df_hatesym_ids['hs'].replace(['sexism', 'racism'],1)

In [127]:
df_hatesym_ids.head()

,id,hs
0,572342978255048705,1
1,572341498827522049,1
2,572340476503724032,1
3,572334712804384768,1
4,572332655397629952,1


In [119]:
for idx, row in df_ruracist.iterrows():
  df_ruracist['TweetID'][idx] = str(idx) + "," + df_ruracist['TweetID'][idx]
df_ruracist.rename(columns = {'Expert':'Amateur'}, inplace = True)
df_ruracist[['TweetID', 'Expert']] = df_ruracist['TweetID'].str.split(',', 1, expand=True)
df_ruracist['Wa_Hov_HS'] = "-"

In [128]:
#Since some tweets are also used from the above dataset, we'll have to check if in both cases those tweets' classification match  
for idx, row in df_hatesym_ids.iterrows():
  if row.id in df_ruracist["TweetID"]:
    df_ruracist['Wa_Hov_HS'][row.id] = row.hs

In [137]:
df_ruracist

,TweetID,Amateur,Amateur_0,Amateur_1,Amateur_2,Amateur_3,Amateur_4,Amateur_5,Amateur_6,Amateur_7,...,Amateur_1082,Amateur_1083,Amateur_1084,Amateur_1085,Amateur_1086,Amateur_1087,Amateur_1088 Amateur_1089,Expert,Wa_Hov_HS,text
597576902212063232,597576902212063232,neither,neither,neither,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,,0
565586175864610817,565586175864610817,neither,neither,neither,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0,0
563881580209246209,563881580209246209,neither,neither,neither,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0,0
595380689534656512,595380689534656512,neither,sexism,neither,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,,0
563757610327748608,563757610327748608,neither,neither,neither,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569176414999588864,569176414999588864,neither,neither,neither,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0,0
571437409843306496,571437409843306496,sexism,sexism,sexism,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sexism,,0
569743990897057792,569743990897057792,neither,neither,neither,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0,0
563027947423272960,563027947423272960,neither,neither,neither,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0,0


In [136]:
df_ruracist['text'] = "0"

In [138]:
for idx, row in df_ruracist.iterrows():
  #This condition is for when execution is stopped and later on resumed
  if not any(c.isalpha() for c in str(df_ruracist['text'][idx])):
    try:
      if api.get_status(df_ruracist['TweetID'][idx]):
        tweetFetched = api.get_status(df_ruracist['TweetID'][idx])
        df_ruracist['text'][idx] = tweetFetched.text
    except tweepy.TweepError as e:
      # print (e.api_code)
      # print (getExceptionMessage(e.reason))
      df_ruracist['text'][idx] = "TWITTER__ERROR" + getExceptionMessage(e.reason)
    #We save the dataframe every line to not lose any progress (in case this cell would stop after 6+ hours!)
    df_ruracist.to_csv('/content/drive/MyDrive/TFG/data/ruracist_data.csv', index=False)  

####Rest of preprocessing


In [121]:
df_ruracist = pd.read_csv('/content/drive/MyDrive/TFG/data/ruracist_data.csv', engine="python")

In [122]:
cols = []
for c in df_ruracist.columns:
  cols.append(c)
unwanted_cols = {'TweetID', 'Expert', 'Wa_Hov_HS', 'text'}
cols = [c for c in cols if c not in unwanted_cols]
uniq_vals = []
for c in cols:
  for v in df_ruracist[c].unique():
    uniq_vals.append(v)

uniq_vals = list(dict.fromkeys(uniq_vals))
#print all possible values in the columns to check them out
print(uniq_vals)

['neither', 'sexism', 'racism', 'both', 'link', nan]


In [123]:
#We replace the values with 0 and 1
#"link" refers to a tweet with a link, we can't use it to determine if it's hate speech or not
df_ruracist["Expert"] = df_ruracist["Expert"].replace(['sexism', 'racism', 'both'],1)
df_ruracist["Expert"] = df_ruracist["Expert"].replace(['neither', 'link'],0)
for c in cols:
  df_ruracist[c] = df_ruracist[c].replace(['sexism', 'racism', 'both'],1)
  df_ruracist[c] = df_ruracist[c].replace(['neither', 'link'],0)

In [124]:
#Amateur_HS will have the overall hs classification of amateur annotators
df_ruracist["Amateur_HS"] = 0
for idx, row in df_ruracist.iterrows():
  for c in cols:
    #When the loop if finished, if Amateur_HS has a negative value the majority said it's not Hate speech and viceversa
    if row[c] == 1:
      df_ruracist["Amateur_HS"][idx] = df_ruracist["Amateur_HS"][idx] + 1
    elif row[c] == 0:
      df_ruracist["Amateur_HS"][idx] = df_ruracist["Amateur_HS"][idx] - 1

In [125]:
#When Amateur_HS is -1, 1 or 0 (a tie), we check the Expert's classification 
#Having all this we can create a Hate speech column with a 0/1 value for this dataset
df_ruracist["hate speech"] = 0
for idx, row in df_ruracist.iterrows():
    if row.Amateur_HS in [0,1,-1]:
      df_ruracist["hate speech"][idx] = df_ruracist["Expert"][idx]
    elif row.Amateur_HS < 0:
      df_ruracist["hate speech"][idx] = 0
    else:
      df_ruracist["hate speech"][idx] = 1

df_ruracist = df_ruracist[["TweetID", "text", "Expert", "Amateur_HS", "hate speech", "Wa_Hov_HS"]]

In [126]:
#This is the dataframe's state: we'll keep these columns for now...
df_ruracist

,TweetID,text,Expert,Amateur_HS,hate speech,Wa_Hov_HS
0,597576902212063232,Cisco had to deal with a fat cash payout to th...,0,-3,0,NaN
1,565586175864610817,"@MadamPlumpette I'm decent at editing, no worr...",0,-3,0,0.0
2,563881580209246209,@girlziplocked will read. gotta go afk for a b...,0,-3,0,0.0
3,595380689534656512,guys. show me the data. show me your github. t...,0,-1,0,NaN
4,563757610327748608,@tpw_rules nothings broken. I was just driving...,0,-3,0,0.0
...,...,...,...,...,...,...
6904,569176414999588864,I used to have pet bunnies. :) I named them P...,0,-3,0,0.0
6905,571437409843306496,TWITTER__ERRORUser has been suspended.,1,3,1,NaN
6906,569743990897057792,@alex SO GROSS. feeling the urge to shower in ...,0,-3,0,0.0
6907,563027947423272960,Purpose of this group is to share the types of...,0,-3,0,0.0


In [127]:
ruracist_data = "ARE YOU RACIST OR AM I SEEING THINGS? DATASET:\n------------------------------------------\n"
ruracist_data = ruracist_data + "Before row elimination:\n"
ruracist_data = ruracist_data + "Number of English tweets: " + str(df_ruracist.shape[0]) + "\n"
ruracist_data = ruracist_data + "Percentage of hate speech: " + str((len(df_ruracist.loc[df_ruracist['hate speech'] == 1].index)*100)/df_ruracist.shape[0]) + "%\n\n"

In [128]:
df_mismatch = df_ruracist[df_ruracist["hate speech"] != df_ruracist["Wa_Hov_HS"]].sort_values(by="Wa_Hov_HS")
df_mismatch = df_mismatch[~df_mismatch['Wa_Hov_HS'].isnull()]
#I'll manually check each mismatched tweet to see If they should be considered hate speech or not (rows with twitter error will be deleted later)
df_mismatch.to_csv('/content/drive/MyDrive/TFG/data/ruracisthatesym_mismatch_data.csv', index=False) 
df_mismatch

,TweetID,text,Expert,Amateur_HS,hate speech,Wa_Hov_HS
65,564904052652113920,TWITTER__ERRORNo status found with that ID.,0,3,1,0.0
4143,563378742152540160,TWITTER__ERRORNo status found with that ID.,0,3,1,0.0
4089,563085113123422209,"... when he hears ""blue haired bitch"", ""gamerg...",1,3,1,0.0
4087,562740608302067712,RT @femfreq: Reminder: In the game of patriarc...,0,3,1,0.0
4078,572338780671107072,Maybe the girls should have less tickets on th...,1,1,1,0.0
...,...,...,...,...,...,...
4981,572321733841063936,TWITTER__ERRORNo status found with that ID.,0,1,0,1.0
5009,572330821433966592,These girls are pretty...awful. #gohome #MKR,1,-3,0,1.0
5373,572334859902844929,STOP SAYING SASS OR I WILL PUT MY FOOT UP YOUR...,0,-3,0,1.0
4167,572338299106291713,Catching up with #MKR. If society judges these...,1,-3,0,1.0


In [129]:
#What I'll do is modify Waseem & Hovy's data directly, so we can delete the duplicated rows from this dataframe directly
df_ruracist = df_ruracist[df_ruracist["Wa_Hov_HS"].isnull()]

In [130]:
ruracist_data = ruracist_data + "After row elimination of overlapping tweets:\n"
ruracist_data = ruracist_data + "Number of English tweets: " + str(df_ruracist.shape[0]) + "\n"
ruracist_data = ruracist_data + "Percentage of hate speech: " + str((len(df_ruracist.loc[df_ruracist['hate speech'] == 1].index)*100)/df_ruracist.shape[0]) + "%\n\n"

In [131]:
for idx, row in df_ruracist.iterrows():
  if "TWITTER__ERROR" in row.text:
    df_ruracist = df_ruracist.drop(labels=idx, axis=0)

In [132]:
ruracist_data = ruracist_data + "After row eliminations (overlap + twitter error):\n"
ruracist_data = ruracist_data + "Number of English tweets: " + str(df_ruracist.shape[0]) + "\n"
ruracist_data = ruracist_data + "Percentage of hate speech: " + str((len(df_ruracist.loc[df_ruracist['hate speech'] == 1].index)*100)/df_ruracist.shape[0]) + "%\n\n"

In [133]:
#When manually checking the overlapped tweets, I saw that many times in this dataset the opinion of the single Expert annotator was more correct than the majority of amateur ones
#In case there's any incorrect classification, I'll manually check the tweets where this disagreement ocurrs
df_ruracist_exp_am_mismatch = df_ruracist[df_ruracist["hate speech"] != df_ruracist["Expert"]].sort_values(by="Wa_Hov_HS")
df_ruracist_exp_am_mismatch.to_csv('/content/drive/MyDrive/TFG/data/ruracis_exp_am_mismatch_data.csv', index=False) 

In [134]:
df_ruracist_exp_am_mismatch

,TweetID,text,Expert,Amateur_HS,hate speech,Wa_Hov_HS
59,594917674448072704,RT @glebd: Ever told a sexist joke and a woman...,0,3,1,NaN
128,603553480754778112,@marvelousmusing I should look and see if ther...,0,3,1,NaN
140,596731670281097218,"@ThelmaSleaze ""i read tweets about it"" she say...",1,-3,0,NaN
157,571899941817331713,"""#feminazi"" 😂😭😭",0,3,1,NaN
191,570639722600538113,con watermelon and grape soda #coon lol RT @Sh...,1,-3,0,NaN
...,...,...,...,...,...,...
6792,596029483791986688,"like sorry dude, but my world doesn't revolve ...",0,3,1,NaN
6817,572221262207979520,"@Videogamehero @TomicComics ""lol what a dumb l...",0,3,1,NaN
6824,575754703071399936,"@BandiBev ""FEMINAZI R ANTI-MEN BECAUSE IT SAYS...",0,3,1,NaN
6858,575672825094258688,"Because i'm a feminazi, that's why...",0,3,1,NaN


In [135]:
#Importing the corrected tweets from "df_ruracist_exp_am_mismatch"
df_ruracis_exp_am_mismatch_data_correct = pd.read_csv('/content/drive/MyDrive/TFG/data/ruracis_exp_am_mismatch_data_correct.csv', engine="python")

In [136]:
#Correcting the classification
for idx1, row1 in df_ruracist.iterrows():
  for idx2, row2 in df_ruracis_exp_am_mismatch_data_correct.iterrows():
    if str(row1.TweetID) == str(row2.TweetID):
      df_ruracist['hate speech'][idx1] = row2['hate speech']

In [137]:
#Finally, we leave the two columns we're interested in
df_ruracist = df_ruracist[["text", "hate speech"]]

In [138]:
df_ruracist

,text,hate speech
0,Cisco had to deal with a fat cash payout to th...,0
3,guys. show me the data. show me your github. t...,0
6,@lysandraws yay! Absolutely. I'm not gone unti...,0
8,@SirenSailor rtfm. http://t.co/jaMXHikl3u,0
10,@Popehat who wouldn't?,0
...,...,...
6891,"Y'know, I get the appeal of group acceptance. ...",0
6893,RT @neilhimself: It looks like by mentioning t...,0
6897,RT @Quinnae_Moon: That's not merely because of...,0
6898,"@katexweis i'm all about black milk. some f21,...",0


In [139]:
ruracist_data = ruracist_data + "After row eliminations and small row correction:\n"
ruracist_data = ruracist_data + "Number of English tweets: " + str(df_ruracist.shape[0]) + "\n"
ruracist_data = ruracist_data + "Percentage of hate speech: " + str((len(df_ruracist.loc[df_ruracist['hate speech'] == 1].index)*100)/df_ruracist.shape[0]) + "%\n\n"

In [140]:
print(ruracist_data)

ARE YOU RACIST OR AM I SEEING THINGS? DATASET:
------------------------------------------
Before row elimination:
Number of English tweets: 6909
Percentage of hate speech: 18.17918656824432%

After row elimination of overlapping tweets:
Number of English tweets: 2790
Percentage of hate speech: 38.63799283154122%

After row eliminations (overlap + twitter error):
Number of English tweets: 2094
Percentage of hate speech: 28.22349570200573%

After row eliminations and small row correction:
Number of English tweets: 2094
Percentage of hate speech: 21.872015281757403%




###[Hate speech dataset from a white supremacist forum ](https://aclanthology.org/W18-5102.pdf)

####Obtaining post text

In [16]:
#In this set we don't have the text or tweet IDs, these posts are from a forum, we have a column with a file id...
#We have the posts' text as text files (loaded in in my drive)
df_whitesup['text'] = ""
for idx, row in df_whitesup.iterrows():
  text = open('/content/drive/MyDrive/TFG/data/all_files_white_sup/' + df_whitesup['file_id'][idx] + '.txt').read()
  df_whitesup['text'][idx] = text

In [18]:
df_whitesup.to_csv('/content/drive/MyDrive/TFG/data/whitesup_data.csv', index=False)  

####Rest of preprocessing


In [141]:
df_whitesup = pd.read_csv('/content/drive/MyDrive/TFG/data/whitesup_data.csv', engine="python")
#We import the corrected data: explanation in a couple of cells below
df_whitesup_correct = pd.read_csv('/content/drive/MyDrive/TFG/data/whitesup_data_correct.csv', encoding='utf8', engine="python")

In [142]:
df_whitesup["label"].unique()

array(['noHate', 'hate', 'idk/skip', 'relation'], dtype=object)

In [53]:
#"relation" refers to posts that alone aren't hateful but with the context, it is"
df_whitesup[df_whitesup["label"]=="relation"].sample(n=10)
#In this sample in some occasions posts are hateful, I want to manually check the posts labelled as "relation" 

,file_id,user_id,subforum_id,num_contexts,label,text
8862,13388443_2,581151,1345,0,relation,Is Africa the epitome of progress or what ?
10181,14022848_1,575397,1388,0,relation,Definitely my dad said in his city growing up ...
8861,13388443_1,581151,1345,0,relation,So we have ox-driven ambulances in Zimbabwe an...
1525,14026532_1,578419,1387,0,relation,Just wait until we open our doors to the poor ...
7044,14039020_1,576887,1387,0,relation,"The work place pensions advert is pretty bad ,..."
4558,13462231_3,582696,1346,0,relation,I feel sorry for whites living in Southern Cal...
4360,31761769_1,599528,1363,0,relation,The most absurd thing I have ever heard a nig ...
2420,13507437_2,575149,1395,0,relation,Soccer and Music .
3558,13553557_2,588238,1346,0,relation,Classic brainwashing .
7284,14651426_2,574041,1371,0,relation,Look what white liberals have done .


In [143]:
df_whitesup = df_whitesup[['text','label']]
df_whitesup.rename(columns = {'label':'hate speech'}, inplace = True)
df_whitesup['hate speech'] = df_whitesup['hate speech'].replace(['noHate'],0)
df_whitesup['hate speech'] = df_whitesup['hate speech'].replace(['hate'],1)
df_whitesup['hate speech'] = df_whitesup['hate speech'].replace(['idk/skip'],"-")

In [144]:
whitesup_data = "WHITE SUPREMACIST FORUM DATASET:\n------------------------------------------\n"
whitesup_data = whitesup_data + "Before row correction and elimination:\n"
whitesup_data = whitesup_data + "Number of English tweets: " + str(df_whitesup.shape[0]) + "\n"
whitesup_data = whitesup_data + "Percentage of hate speech: " + str((len(df_whitesup.loc[df_whitesup['hate speech'] == 1].index)*100)/df_whitesup.shape[0]) + "%\n\n"

In [145]:
#Deleting all rows labelled as "idk/skip": 
# "Sentences that are not written in English or that
# do not contain information as to be classified into
# HATE or NOHATE are given this label."
df_whitesup = df_whitesup[df_whitesup['hate speech'] != "-"]

In [146]:
whitesup_data = whitesup_data + "After row elimination:\n"
whitesup_data = whitesup_data + "Number of English tweets: " + str(df_whitesup.shape[0]) + "\n"
whitesup_data = whitesup_data + "Percentage of hate speech: " + str((len(df_whitesup.loc[df_whitesup['hate speech'] == 1].index)*100)/df_whitesup.shape[0]) + "%\n\n"

In [42]:
df_whitesup.to_csv('/content/drive/MyDrive/TFG/data/whitesup_data_to_correct.csv', index=False)  

In [147]:
df_whitesup_correct

,text,hate speech
0,You are telling me that 78 % of kids under 6 i...,0
1,Jeeze its worst than the UK .,0
2,So far today I 've come across the words scum ...,0
3,All too describe my sisters .,0
4,The Indian teenager was showing the Caucasian ...,0
...,...,...
10866,This is after a famous incident of former Prim...,0
10867,"Billy - `` That guy would n't leave me alone ,...",0
10868,Wish we at least had a Marine Le Pen to vote f...,0
10869,Its like the choices are white genocide candid...,0


In [150]:
df_whitesup = df_whitesup_correct

In [148]:
whitesup_data = whitesup_data + "After row correction:\n"
whitesup_data = whitesup_data + "Number of English tweets: " + str(df_whitesup_correct.shape[0]) + "\n"
whitesup_data = whitesup_data + "Percentage of hate speech: " + str((len(df_whitesup_correct.loc[df_whitesup_correct['hate speech'] == 1].index)*100)/df_whitesup_correct.shape[0]) + "%\n\n"

In [149]:
print(whitesup_data)

WHITE SUPREMACIST FORUM DATASET:
------------------------------------------
Before row correction and elimination:
Number of English tweets: 10944
Percentage of hate speech: 10.928362573099415%

After row elimination:
Number of English tweets: 10871
Percentage of hate speech: 11.001747769294454%

After row correction:
Number of English tweets: 10871
Percentage of hate speech: 11.286910127863122%




###[The Gab Hate Corpus](https://osf.io/edua3/)

In [151]:
df_ghc_data = "THE GAB HATE CORPUS DATASET:\n------------------------------------------\n"

In [152]:
df_ghc_data = df_ghc_data + "Gab hate corpus train set:\n"
df_ghc_data = df_ghc_data + "Number of English tweets: " + str(df_ghc_train.shape[0]) + "\n\n"
df_ghc_train = df_ghc_train[['text','hd']]
df_ghc_train.rename(columns = {'hd':'hate speech'}, inplace = True)

In [153]:
df_ghc_data = df_ghc_data + "Gab hate corpus test set:\n"
df_ghc_data = df_ghc_data + "Number of English tweets: " + str(df_ghc_test.shape[0]) + "\n\n"
df_ghc_test = df_ghc_test[['text','hd']]
df_ghc_test.rename(columns = {'hd':'hate speech'}, inplace = True)

In [154]:
df_ghc = pd.concat([df_ghc_train, df_ghc_test])

In [155]:
df_ghc

,text,hate speech
0,He most likely converted to islam due to his n...,0
1,So Ford lied about being a psychologist. Recor...,0
2,Jobs. Education. Ending abuse of Nation. CA43.,0
3,"I share a lot of your values, & like many who ...",0
4,I am so ready to get back to blogging! www.ben...,0
...,...,...
5505,"Trump To ""Counter"" DNC Lawsuit; Seeks Servers,...",0
5506,i guess eu is gonna have to back track a littl...,0
5507,A good read here.... https://countrysquire....,0
5508,The only way to change things is to have compa...,0


In [15]:
df_ghc_data = df_ghc_data + "Gab hate corpus all posts:\n"
df_ghc_data = df_ghc_data + "Number of English tweets: " + str(df_ghc.shape[0]) + "\n"
df_ghc_data = df_ghc_data + "Percentage of hate speech: " + str((len(df_ghc.loc[df_ghc['hate speech'] == 1].index)*100)/df_ghc.shape[0]) + "%\n"
print(df_ghc_data)

THE GAB HATE CORPUS DATASET:
------------------------------------------
Gab hate corpus train set:
Number of English tweets: 22036

Gab hate corpus test set:
Number of English tweets: 5510

Gab hate corpus all posts:
Number of English tweets: 27546
Percentage of hate speech: 8.523923618674218%



###Unification, hashtags and emojis

In [156]:
df = pd.concat([df_hateval, df_exist, df_aut_hate, df_hatesym, df_ruracist, df_whitesup, df_ghc])

In [157]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data.csv', index=False)

In [35]:
df = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/unfinished_english_data.csv', encoding='utf8', engine='python')

####Emoji code extraction and replacement

In [31]:
emoji_codes = getAllEmojiHexCode(df)

In [32]:
emoji_codes

array(['&#10004;', '&#10006;', '&#10024;', '&#10052;', '&#10060;',
       '&#10067;', '&#10068;', '&#10069;', '&#10071;', '&#10084;',
       '&#1040784;', '&#1040788;', '&#1041184;', '&#1041185;',
       '&#1041190;', '&#1041191;', '&#1041196;', '&#1041204;',
       '&#1041205;', '&#1041210;', '&#1041216;', '&#1041219;',
       '&#1041222;', '&#1041223;', '&#1041233;', '&#1041237;',
       '&#1041240;', '&#1041243;', '&#1041629;', '&#1041636;',
       '&#1041653;', '&#1041681;', '&#1042390;', '&#1042469;',
       '&#1043214;', '&#1043215;', '&#1043291;', '&#1043292;',
       '&#1043293;', '&#1043323;', '&#1043350;', '&#1043351;',
       '&#1043353;', '&#1043357;', '&#1043358;', '&#1043359;',
       '&#1043360;', '&#1043473;', '&#11015;', '&#11088;', '&#11093;',
       '&#12288;', '&#12290;', '&#127344;', '&#127378;', '&#127379;',
       '&#127382;', '&#127383;', '&#127386;', '&#127463;', '&#127468;',
       '&#127470;', '&#127480;', '&#127481;', '&#127482;', '&#127744;',
       '&#1277

In [41]:
df = replaceEmojiCodes(df)

####Hashtag and emoji filtering

In [40]:
df, hashtag_dict, emoji_dict = cleanTweets_getDicts(df)

In [ ]:
hashtag_dict_important = important_values(hashtag_dict, 30)
emoji_dict_important = important_values(emoji_dict, 30)

In [ ]:
hashtag_dict_important

In [ ]:
emoji_dict_important

In [37]:
print("HATE SPEECH - PERCENTAGE OF ALL ENGLISH POSTS:")
print("WITH HATE SPEECH: " + str((len(df.loc[df['hate speech'] == 1].index)*100)/df.shape[0]))
print("WITHOUT HATE SPEECH: " + str((len(df.loc[df['hate speech'] == 0].index)*100)/df.shape[0]))

HATE SPEECH - PERCENTAGE OF ALL ENGLISH POSTS:
WITH HATE SPEECH: 17.429390528774686
WITHOUT HATE SPEECH: 82.56742408154598


#####Removal of all emojis and hashtags

In [ ]:
#Remove emojis
df = replace_emojis([], list(emoji_dict.keys()), df)

In [ ]:
#Remove hashtags
for idx, row in df.iterrows():
  for k in list(hashtag_dict.keys()):
      row.text = row.text.replace(k, "")
      df['text'][idx] = row.text

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/removed_english_data.csv', index=False)  

#####Filtered emojis and hashtags

In [ ]:
df = replace_emojis()

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/filtered_english_data.csv', index=False)  

#####All emojis and hashtags kept

In [ ]:
#Replace emojis with text
df = replace_emojis(list(emoji_dict.keys()), [], df)

In [ ]:
df.to_csv('/content/drive/MyDrive/TFG/data/final_data/mantained_english_data.csv', index=False)  

#DISCARDED DATA

In [ ]:
#Large Scale Crowdsourcing and Characterization of Twitter Abusive Behavior
df_abusive_text = pd.read_csv('/content/hatespeech_id_label_founta.csv', encoding='utf8', header= None)

###[Large Scale Crowdsourcing and Characterization of Twitter Abusive Behavior](https://arxiv.org/pdf/1802.00393.pdf)

####Obtaining tweet text

In [ ]:
#Split the single column into 2
df_abusive_text[['TweetID', 'hate speech']] = df_abusive_text[0].str.split(',', 1, expand=True)
df_abusive_text = df_abusive_text[['TweetID', 'hate speech']]

In [ ]:
df_abusive_text['text'] = "0"

In [ ]:
df_abusive_text = pd.read_csv('/content/drive/MyDrive/TFG/data/abusive_text_data.csv', encoding='utf8', engine="python")

In [ ]:
df_abusive_text

,TweetID,hate speech,text
0,849667487180259329,abusive,TWITTER__ERRORNo status found with that ID.
1,850490912954351616,abusive,Alex Brosas another idiot #ALDUBKSGoesToUS ht...
2,848791766853668864,abusive,"RT @ItIzBiz: as Nancy Reagan would say, 'just ..."
3,848306464892604416,abusive,TWITTER__ERRORNo status found with that ID.
4,850010509969465344,normal,RT @MailOnline: The Nazi death gas so horrific...
...,...,...,...
100002,848922176128376832,hateful,0
100003,849405078956109829,abusive,0
100004,847981251143254016,normal,0
100005,850180983253073920,abusive,0


In [ ]:
for idx, row in df_abusive_text.iterrows():
  if "TWITTER__ERROR" in str(row.text) or "0" in str(row.text) or row["hate speech"]!="hateful":
    df_abusive_text = df_abusive_text.drop(labels=idx, axis=0)

In [ ]:
df_abusive_text

,TweetID,hate speech,text
5,850433664890544128,hateful,I hate er chase because if the Bitch that work...
12,850100742035836929,hateful,RT @nyctophil3: Pineapples do not belong on pi...
21,847804507367100416,hateful,Niggas keep talking about women wearing weave ...
33,850090319165050880,hateful,@vappywave idiot that's not gonna work. you go...
67,848686686930382848,hateful,RT @ayevonnn: bruh i fucking hate people like ...
...,...,...,...
44055,849005550524387328,hateful,lrt is cute because they're opposite elements ...
44088,850157792925241346,hateful,Voted for the biggest fucking liar @POTUS ever...
44348,848578067043889153,hateful,"Supposed ""Anti-fascist"" favors murderous commu..."
44371,850378195224465411,hateful,Syrian ally Iran blasts U.S. missile strikes a...


In [ ]:
for idx, row in df_abusive_text.iterrows():
  #This condition is for when execution is stopped and later on resumed
  if not any(c.isalpha() for c in str(df_abusive_text['text'][idx])):
    try:
      if api.get_status(df_abusive_text['TweetID'][idx]):
        tweetFetched = api.get_status(df_abusive_text['TweetID'][idx])
        df_abusive_text['text'][idx] = tweetFetched.text
    except tweepy.TweepError as e:
      # print (e.api_code)
      # print (getExceptionMessage(e.reason))
      df_abusive_text['text'][idx] = "TWITTER__ERROR" + getExceptionMessage(e.reason)
    #We save the dataframe every line to not lose any progress (in case this cell would stop after 6+ hours!)
    df_abusive_text.to_csv('/content/drive/MyDrive/TFG/data/abusive_text_data.csv', index=False)  

####Rest of preprocessing


In [ ]:
df_abusive_text['hate speech'].unique()

array(['abusive', 'normal', 'hateful', 'spam'], dtype=object)